In [10]:
import pandas as pd
from google.colab import auth
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig, SchemaField
from datetime import datetime

auth.authenticate_user()
print("Authenticated with Google Cloud!")

PROJECT_ID = 'excellent-badge-443909-a7'
DATASET_ID = 'customer_data'
CUSTOMER_TABLE_ID = 'Customer_dimension'
ORDER_TABLE_ID = 'Order_dimension'
CSV_FILE_PATH = '/content/Customer_orders.csv'
CSV_FILE2_PATH = '/content/Customer_orders1.csv'

client = bigquery.Client(project=PROJECT_ID)

dataset_ref = client.dataset(DATASET_ID)
dataset = bigquery.Dataset(dataset_ref)
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Dataset {DATASET_ID} created or already exists.")


df = pd.read_csv(CSV_FILE2_PATH)

print("Initial DataFrame info:")
df.info()
print("\nSample data:")
print(df.head())

# Step 2: Data Transformations

# Remove duplicates
df = df.drop_duplicates()
print("\nDuplicates removed.")

# Null handling:
# - For demonstration: Remove rows where CustId or orderid is null (critical keys)
df = df.dropna(subset=['CustId', 'orderid'])
print("Rows with null in CustId or orderid removed.")

# - Replace nulls in other columns:
#   - Custname: Replace with 'Unknown'
#   - location: Replace with 'Unknown'
#   - orderprice: Replace with 0
#   - ordqty: Replace with 0
df['Custname'] = df['Custname'].fillna('Unknown')
df['location'] = df['location'].fillna('Unknown')
df['orderprice'] = df['orderprice'].fillna(0)
df['ordqty'] = df['ordqty'].fillna(0)
print("Nulls replaced in non-key columns.")

# Add new column: ordered_datetime (using current datetime for each row)
df['ordered_datetime'] = datetime.now().isoformat()
print("Added 'ordered_datetime' column.")

# Print transformed DataFrame info
print("\nTransformed DataFrame info:")
df.info()
print("\nSample transformed data:")
print(df.head())

# Step 3: Split into Dimension Tables

# Customer_dimension: Unique customers (CustId, Custname, location)
customer_df = df[['CustId', 'Custname', 'location']].drop_duplicates().reset_index(drop=True)

# Order_dimension: Orders (orderid, orderprice, ordqty, ordered_datetime, CustId as FK)
order_df = df[['orderid', 'orderprice', 'ordqty', 'ordered_datetime', 'CustId']].reset_index(drop=True)

# Step 4: Load to BigQuery

# Define schemas
customer_schema = [
    SchemaField("CustId", "INTEGER", mode="REQUIRED"),
    SchemaField("Custname", "STRING", mode="NULLABLE"),
    SchemaField("location", "STRING", mode="NULLABLE"),
]

order_schema = [
    SchemaField("orderid", "INTEGER", mode="REQUIRED"),
    SchemaField("orderprice", "FLOAT", mode="NULLABLE"),
    SchemaField("ordqty", "INTEGER", mode="NULLABLE"),
    SchemaField("ordered_datetime", "STRING", mode="NULLABLE"),  # Stored as ISO string
    SchemaField("CustId", "INTEGER", mode="REQUIRED"),
]

# Load Customer_dimension
customer_table_ref = dataset.table(CUSTOMER_TABLE_ID)
customer_job_config = LoadJobConfig(
    schema=customer_schema,
    write_disposition="WRITE_TRUNCATE"  # Overwrite table if exists
)
customer_job = client.load_table_from_dataframe(
    customer_df, customer_table_ref, job_config=customer_job_config
)
customer_job.result()  # Wait for job to complete
print(f"Loaded {customer_job.output_rows} rows into {CUSTOMER_TABLE_ID}.")

# Load Order_dimension
order_table_ref = dataset.table(ORDER_TABLE_ID)
order_job_config = LoadJobConfig(
    schema=order_schema,
    write_disposition="WRITE_TRUNCATE"  # Overwrite table if exists
)
order_job = client.load_table_from_dataframe(
    order_df, order_table_ref, job_config=order_job_config
)
order_job.result()  # Wait for job to complete
print(f"Loaded {order_job.output_rows} rows into {ORDER_TABLE_ID}.")

print("Pipeline completed successfully.")

Authenticated with Google Cloud!
Dataset customer_data created or already exists.
Initial DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustId      100 non-null    int64  
 1   Custname    100 non-null    object 
 2   location    100 non-null    object 
 3   orderid     100 non-null    int64  
 4   orderprice  100 non-null    float64
 5   ordqty      100 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 4.8+ KB

Sample data:
   CustId        Custname     location  orderid  orderprice  ordqty
0     121     Bob Johnson       Dallas     1001      374.89       3
1     124    Bob Martinez      Seattle     1002      139.54       7
2     102     Alice Smith  San Antonio     1003      356.66      10
3     101       Ivy Brown      Seattle     1004      635.08       8
4     119  Emma Rodriguez     New York     1005